In [1]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import openai
import os
from openai import OpenAI
import time
from google.colab import userdata
import pandas as pd
import os

In [4]:
key= userdata.get('OpenAIkey')

In [5]:
os.environ['OPENAI_API_KEY'] = key
client = OpenAI()

In [8]:
def get_stance(text, target):
    prompt = f"Based solely on the given text: \"{text}\". What is the stance expressed towards {target} in this text? Please select from 'Favour','Against', or 'Neutral'"
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}]

    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Specify the chat model
            messages=messages
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        if "rate limit" in str(e).lower():
            print("Rate limit reached. Waiting before retrying...")
            time.sleep(60)  # Wait for 60 seconds before retrying
            return get_stance(text, target)  # Retry
        else:
            raise

In [9]:
def read_article(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

English and Hindi Stance

In [30]:
df = pd.read_csv("/content/drive/MyDrive/thesis-data/all_common_entities_af_hi.csv")
df

,File Name,Common Entity,Count
0,Q15860072.txt,Russia,3
1,Q15860072.txt,Ukraine,3
2,Q110999040.txt,Russia,11
3,Q110999040.txt,Ukraine,7
4,Q110999040.txt,Belarus,2
...,...,...,...
248,Q57282.txt,Zuma,2
249,Q57282.txt,the African National Congress,2
250,Q7888194.txt,United Kingdom,6
251,Q7888194.txt,European Union,11


In [31]:
import pandas as pd


# Iterate over each row in the dataset
for index, row in df.iterrows():
    file_name = row['File Name']
    entity = row['Common Entity']

    # Construct file paths for English and Hindi articles
    eng_article_path = f"/content/drive/MyDrive/thesis-data/Articles_Translated_SD/Afrikaans_Articles/{file_name}"
    hin_article_path = f"/content/drive/MyDrive/thesis-data/Articles_Translated_SD/Hindi_Articles/{file_name}"

    # Read the articles
    eng_article = read_article(eng_article_path)
    hin_article = read_article(hin_article_path)

    # Get stance using GPT API
    stance_eng = get_stance(eng_article, entity)
    stance_hin = get_stance(hin_article, entity)

    # Update the DataFrame
    df.at[index, 'Stance-Af'] = stance_eng
    df.at[index, 'Stance-Hin'] = stance_hin

# Define the mapping function
def map_stance(stance):
    if "Favour" in stance:
        return 1
    elif "Neutral" in stance:
        return 0
    elif "Against" in stance:
        return -1
    else:
        return 0  # Or an appropriate value for stances not covered

# Apply the transformation
df['Stance-Af'] = df['Stance-Af'].apply(map_stance)
df['Stance-Hin'] = df['Stance-Hin'].apply(map_stance)

# Save the modified DataFrame
df.to_csv("/content/drive/MyDrive/thesis-data/stance_results/final_common_entities_af_hi.csv", index=False)


In [16]:
def needs_processing(stance):
    # Check if the stance is empty, None, or a specific placeholder (e.g., 'NA')
    return pd.isna(stance) or stance in ["", "NA"]

In [15]:
df.to_csv("/content/drive/MyDrive/thesis-data/stance_results/final_common_entities_eng_hi.csv")

In [14]:
df = pd.read_csv("/content/drive/MyDrive/thesis-data/final_common_entities_zh_hi.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/thesis-data/final_common_entities_zh_hi.csv'

In [ ]:
df.columns = [*df.columns[:-2], 'Stance-Zh', 'Stance-Hin']
df

,File Name,Common Entity,Count,Stance-zh-h,Stance-hi-h,Stance-Zh,Stance-Hin
0,Q484260.txt,Guru,3,0,1,1,0
1,Q1439.txt,Texas,6,0,0,0,0
2,Q1439.txt,United States,2,0,0,0,0
3,Q126065.txt,Iran,4,0,0,0,0


In [ ]:
df.to_csv("/content/drive/MyDrive/thesis-data/final_common_entities_zh_hi.csv", index=False)

In [ ]:
import pandas as pd
import os

# Directory containing the CSV files
directory = '/content/drive/MyDrive/thesis-data/Accuracy'

# Languages to consider
languages = ['zh', 'af', 'hin', 'eng']

# Dictionary to hold accuracy for each language
accuracy_dict = {lang: [] for lang in languages}

# Read each CSV file and calculate accuracies
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        try:
            df = pd.read_csv(filepath)


            for lang in languages:
                col_h = f'Stance-{lang}-h'
                col = f'Stance-{lang.capitalize()}'


                if col_h in df.columns and col in df.columns:
                    matches = df[col_h] == df[col]
                    accuracy = matches.sum() / len(df)
                    accuracy_dict[lang].append(accuracy)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")


for lang, accuracies in accuracy_dict.items():
    if accuracies:
        average_accuracy = sum(accuracies) / len(accuracies)
        print(f"Average accuracy for {lang}: {average_accuracy:.2f}")
    else:
        print(f"No data to calculate accuracy for {lang}")


Average accuracy for zh: 0.77
Average accuracy for af: 0.75
Average accuracy for hin: 1.00
Average accuracy for eng: 0.59
